# Análisis Temporal de Tiempos de Inactividad

Este notebook se enfoca en el análisis de patrones temporales y tendencias en los datos de tiempo de inactividad.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import sys
sys.path.append('..')

from utils.db_connection import create_connection, read_table
from utils.visualization import plot_downtime_distribution
from utils.statistics import calculate_basic_stats

%matplotlib inline
plt.style.use('seaborn')

## 1. Carga y Preparación de Datos

In [ ]:
# Cargar datos
conn = create_connection()
production_data = read_table(conn, 'manufacturing_line_productivity')
downtime_data = read_table(conn, 'normalizeddowntime')

# Preparar datos temporales
production_data['date'] = pd.to_datetime(production_data['date'])
daily_downtime = downtime_data.groupby('batch').agg({
    'downtime_minutes': 'sum'
}).reset_index()

# Unir con fechas de producción
temporal_data = pd.merge(daily_downtime, production_data[['batch', 'date']], on='batch')
temporal_data = temporal_data.sort_values('date')

## 2. Análisis de Tendencias Diarias

In [ ]:
# Agregación diaria
daily_avg = temporal_data.groupby('date')['downtime_minutes'].agg(['mean', 'std']).reset_index()

# Gráfico de tendencia diaria
plt.figure(figsize=(15, 6))
plt.plot(daily_avg['date'], daily_avg['mean'], label='Promedio Diario')
plt.fill_between(daily_avg['date'],
                 daily_avg['mean'] - daily_avg['std'],
                 daily_avg['mean'] + daily_avg['std'],
                 alpha=0.2)
plt.title('Tendencia Diaria de Tiempo de Inactividad')
plt.xlabel('Fecha')
plt.ylabel('Minutos de Inactividad')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig('../visualizations/daily_trend.png')
plt.show()

## 3. Análisis de Patrones Semanales

In [ ]:
# Agregar día de la semana
temporal_data['weekday'] = temporal_data['date'].dt.day_name()

# Análisis por día de la semana
weekday_stats = temporal_data.groupby('weekday')['downtime_minutes'].agg(['mean', 'std']).reset_index()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_stats['weekday'] = pd.Categorical(weekday_stats['weekday'], categories=weekday_order, ordered=True)
weekday_stats = weekday_stats.sort_values('weekday')

plt.figure(figsize=(12, 6))
plt.bar(weekday_stats['weekday'], weekday_stats['mean'], yerr=weekday_stats['std'], capsize=5)
plt.title('Tiempo de Inactividad Promedio por Día de la Semana')
plt.xlabel('Día de la Semana')
plt.ylabel('Minutos de Inactividad')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../visualizations/weekday_pattern.png')
plt.show()

## 4. Análisis de Estacionalidad

In [ ]:
# Preparar datos para descomposición
daily_series = temporal_data.groupby('date')['downtime_minutes'].mean()
daily_series = daily_series.asfreq('D')

# Descomposición de series temporales
decomposition = seasonal_decompose(daily_series, period=7, model='additive')

# Visualizar componentes
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(15, 12))

decomposition.observed.plot(ax=ax1)
ax1.set_title('Observado')

decomposition.trend.plot(ax=ax2)
ax2.set_title('Tendencia')

decomposition.seasonal.plot(ax=ax3)
ax3.set_title('Estacional')

decomposition.resid.plot(ax=ax4)
ax4.set_title('Residual')

plt.tight_layout()
plt.savefig('../visualizations/seasonal_decomposition.png')
plt.show()

## 5. Análisis de Autocorrelación

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Calcular y visualizar autocorrelación
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

plot_acf(daily_series.dropna(), ax=ax1, lags=30)
ax1.set_title('Función de Autocorrelación (ACF)')

plot_pacf(daily_series.dropna(), ax=ax2, lags=30)
ax2.set_title('Función de Autocorrelación Parcial (PACF)')

plt.tight_layout()
plt.savefig('../visualizations/autocorrelation.png')
plt.show()

## 6. Análisis de Cambios Significativos

In [ ]:
# Calcular cambios significativos
daily_avg['rolling_mean'] = daily_avg['mean'].rolling(window=7).mean()
daily_avg['rolling_std'] = daily_avg['mean'].rolling(window=7).std()

# Detectar valores atípicos (más de 2 desviaciones estándar)
threshold = 2
daily_avg['is_outlier'] = abs(daily_avg['mean'] - daily_avg['rolling_mean']) > (threshold * daily_avg['rolling_std'])

# Visualizar cambios significativos
plt.figure(figsize=(15, 6))
plt.plot(daily_avg['date'], daily_avg['mean'], label='Valor Diario', alpha=0.6)
plt.plot(daily_avg['date'], daily_avg['rolling_mean'], label='Media Móvil (7 días)', linewidth=2)

# Marcar valores atípicos
outliers = daily_avg[daily_avg['is_outlier']]
plt.scatter(outliers['date'], outliers['mean'], color='red', label='Valores Atípicos', zorder=5)

plt.title('Detección de Cambios Significativos en Tiempo de Inactividad')
plt.xlabel('Fecha')
plt.ylabel('Minutos de Inactividad')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../visualizations/significant_changes.png')
plt.show()

## 7. Conclusiones del Análisis Temporal

1. Tendencias Generales:
   - [Completar con observaciones sobre tendencias identificadas]

2. Patrones Semanales:
   - [Completar con patrones semanales identificados]

3. Estacionalidad:
   - [Completar con hallazgos sobre estacionalidad]

4. Cambios Significativos:
   - [Completar con análisis de cambios importantes]

## Recomendaciones

1. [Completar con recomendaciones basadas en patrones temporales]
2. [Completar con sugerencias para mejora]
3. [Completar con estrategias de prevención]

## Próximos Pasos

1. Desarrollar modelos predictivos basados en patrones temporales
2. Investigar causas de valores atípicos identificados
3. Implementar sistema de alertas tempranas
4. Evaluar efectividad de medidas correctivas